In [1]:
!pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
import fastbook
fastbook.setup_book()

In [2]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 168 kB of archives.
After this operation, 567 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 unzip amd64 6.0-21ubuntu1.1 [168 kB]
Fetched 168 kB in 0s (405 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 15734 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-21ubuntu1.1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 17%] [#########.................................................] 8Unpacking unzip (6.0-21ubuntu1.1) ...
7Progress: [ 33%] [###################............................

In [3]:
from fastbook import *

In [9]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
cred_path

Path('/root/.kaggle/kaggle.json')

In [10]:
import os

if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    os.system("cp -n kaggle.json ~/.kaggle")
    cred_path.chmod(0o600)

In [11]:
#hide
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [12]:
from fastai.tabular.all import *

In [13]:
data_path = Path('/storage/data/titanic')
data_path.exists()

True

In [14]:
if not data_path.exists():
    os.system("mkdir /storage/data/titanic")

In [10]:
csv_names = ('train.csv', 'test.csv', 'gender_submission.csv')
for csv in csv_names:
    path = data_path/csv
    if not path.exists():
        os.system('kaggle competitions download -c titanic')
        os.system('mv titanic.zip /storage/data/titanic')
        os.system('unzip /storage/data/titanic/titanic.zip')
        os.system('rm /storage/data/titanic/titanic.zip')
        
for csv in csv_names:
    assert (data_path/csv).exists()

In [10]:
def r_mse(pred, y): return round(math.sqrt(((pred-y)**2).mean()), 6)

In [15]:
def error(pred, y):
    return torch.where(pred != y)[0].size()[0] / pred.size()[0]

In [16]:
df = pd.read_csv(data_path/'train.csv', low_memory=False)
df.head()

,PassengerId,Survived,Pclass,Name,...,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",...,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",...,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",...,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",...,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",...,373450,8.0500,NaN,S


In [18]:
df['Sex'] = df['Sex'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')
df['Survived'] = df['Survived'].astype('category')
df['Pclass'] = df['Pclass'].astype(pd.CategoricalDtype(categories=[1,2,3], ordered=True))

In [19]:
df['Sex'].cat.categories, df['Embarked'].cat.categories

(Index(['female', 'male'], dtype='object'),
 Index(['C', 'Q', 'S'], dtype='object'))

In [20]:
df.dtypes

PassengerId       int64
Survived       category
Pclass         category
Name             object
Sex            category
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked       category
dtype: object

In [21]:
rf = RandomForestClassifier(n_jobs=-1, min_samples_leaf=30, max_samples=100, max_features=3)

In [22]:
cont,cat = cont_cat_split(df)
cont,cat

(['PassengerId', 'Age', 'Fare'],
 ['Survived',
  'Pclass',
  'Name',
  'Sex',
  'SibSp',
  'Parch',
  'Ticket',
  'Cabin',
  'Embarked'])

In [23]:
procs = [Categorify, FillMissing]

In [24]:
train,valid = RandomSplitter(seed=42)(df)

In [25]:
dl = TabularPandas(
    df,
    procs,
    cat_names=cat,
    cont_names=cont,
    y_names="Survived",
    splits=(train,valid),
    reduce_memory=False)

In [26]:
dl.train.xs

,Survived,Pclass,Name,Sex,...,Age_na,PassengerId,Age,Fare
788,1,3,167,2,...,1,789,1.0,20.5750
525,0,3,197,2,...,1,526,40.5,7.7500
821,1,3,393,2,...,1,822,27.0,8.6625
253,0,3,388,2,...,1,254,30.0,16.1000
374,0,3,503,1,...,1,375,3.0,21.0750
...,...,...,...,...,...,...,...,...,...
173,0,3,601,2,...,1,174,21.0,7.9250
462,0,1,227,2,...,1,463,47.0,38.5000
358,1,3,417,1,...,2,359,28.0,7.8792
532,0,3,189,2,...,1,533,17.0,7.2292


In [27]:
rf.fit(dl.train.xs, dl.train.ys)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(max_features=3, max_samples=100, min_samples_leaf=30,
                       n_jobs=-1)

In [28]:
predicted_train_ys = tensor(rf.predict(dl.train.xs))
train_error = error(predicted_train_ys, tensor(dl.train.ys).squeeze() )
# train_rmse = r_mse(predicted_train_ys, dl.train.ys)
predicted_valid_ys = tensor(rf.predict(dl.valid.xs))
valid_error = error(predicted_valid_ys, tensor(dl.valid.ys).squeeze() )
# valid_rmse = r_mse(predicted_valid_ys, dl.valid.ys)
train_error, valid_error

(0.030855539971949508, 0.0056179775280898875)

In [88]:
(torch.where(predicted_train_ys[:5] != tensor([1,0,0,0,1]))[0]).size()[0]

2

In [39]:
train_rmse,valid_rmse

NameError: name 'train_rmse' is not defined